<a href="https://colab.research.google.com/github/silverstar0727/n_body_problem/blob/master/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

20y 07m 19d

In [276]:
import numpy as np
import math

In [277]:
# initial condition
r_0 = [[0.23451, 3450.12, 0345.12], [1.1345, 0345.11, 0.13451], [5345.5345, 0.1345, 0345.1]] # [m]
v_0 = [[-1, 1, 2], [45, 67, 32], [123, 345,235]] # [m/s]

m1 = 1.989e30 # [m]
m2 = 5.972e24
m3 = 1.898e27
m = [m1, m2, m3]

In [278]:
# constant
G = 6.6726e-11 # [Nm2/kg2]

![](https://upload.wikimedia.org/wikipedia/commons/b/bc/Spherical_coordinate.gif)

In [279]:
# 물체 1,2
def force12(r1, r2): # 물체 1과 2의 위치를 r1, r2의 직교좌표계 리스트로 받음
  F = np.zeros(3)
  r12 = np.zeros(3)

  r12[0] = r1[0] - r2[0]
  r12[1] = r1[1] - r2[1]
  r12[2] = r1[2] - r2[2]

  Fmag = G*m1*m2 / (np.linalg.norm(r12)**2) # 힘의크기
  theta = np.arcsin(r12[2] / np.linalg.norm(r12)) # theta
  pi = np.arctan(np.abs(r12[1]/np.abs(r12[0]))) # pi

  # Force
  F[0] = Fmag * np.cos(theta) * np.cos(pi) 
  F[1] = Fmag * np.cos(theta) * np.sin(pi)
  F[2] = Fmag * np.sin(theta)

  return F

def force23(r2, r3):
  F = np.zeros(3)
  r23 = np.zeros(3)
  
  r23[0] = r2[0] - r3[0]
  r23[1] = r2[1] - r3[1]
  r23[2] = r2[2] - r3[2]

  Fmag = G*m2*m3 / (np.linalg.norm(r23)**2)
  theta = np.arcsin(r23[2] / np.linalg.norm(r23))
  pi = np.arctan(np.abs(r23[1]/np.abs(r23[0])))

  F[0] = Fmag * np.cos(theta) * np.cos(pi)
  F[1] = Fmag * np.cos(theta) * np.sin(pi)
  F[2] = Fmag * np.sin(theta)

  return F

def force13(r1, r3):
  F = np.zeros(3)
  r13 = np.zeros(3)
  
  r13[0] = r1[0] - r3[0]
  r13[1] = r1[1] - r3[1]
  r13[2] = r1[2] - r3[2]

  Fmag = G*m1*m3 / (np.linalg.norm(r13)**2)
  theta = np.arcsin(r13[2] / np.linalg.norm(r13))
  pi = np.arctan(np.abs(r13[1]/np.abs(r13[0])))

  F[0] = Fmag * np.cos(theta) * np.cos(pi)
  F[1] = Fmag * np.cos(theta) * np.sin(pi)
  F[2] = Fmag * np.sin(theta)

  return F

# 타겟 물체에 대한 net force
def force(object_num, r0):
  # 타겟 물체를 각각 1,2,3의 정수로 object_num인자에 받고, 초기위치를 직교좌표계로 r_0에 3x3개 좌표로받음
  if object_num == 1:
    return force12(r0[0], r0[1]) + force13(r0[0], r0[2])
  
  elif object_num == 2:
    return -force12(r0[0], r0[1]) + force23(r0[1], r0[2])
  
  elif object_num == 3:
    return -force13(r0[0],r0[2]) - force23(r0[1], r0[2])
  
  else:
    print('object_num error')

In [280]:
# 2계 ODE를 2개의 1계 ODE의 연립으로 변환
def dr_dt(t, r0, v0):
  return v0

def dv_dt(t, r0, v0):
  result = []

  for i in range(3):
    result.append(force(i + 1, r0)/m[i])

  return result

# RK4 method를 이용한 numerical 풀이
def RK4(t, r0, v0, h ): # r0, v0 = [[1의 r,v], [2의 r,v], [3의 r,v]]
  r0 = np.array(r0)
  v0 = np.array(v0)

  a = h*np.array([dr_dt(t, r0, v0),
                  dv_dt(t, r0, v0)])
  b = h*np.array([dr_dt(t + 0.5*h, 0, v0 + 0.5*h*a[1]),
                  dv_dt(t + 0.5*h, r0 + 0.5*h*a[0], 0)])
  c = h*np.array([dr_dt(t + 0.5*h, 0, v0 + 0.5*h*b[1]),
                  dv_dt(t + 0.5*h, r0 + 0.5*h*b[0], 0)])
  d = h*np.array([dr_dt(t + h, 0, v0 + h*c[1]),
                  dv_dt(t + h, r0 + h*c[0], 0)])
  
  
  r0 = (1/6)*(a[0] + 2*b[0] + 2*c[0] + d[0])
  v0 = (1/6)*(a[1] + 2*b[1] + 2*c[1] + d[1])
  
  return [r0, v0]

In [281]:
h = 0.1 # time step size
t = 0  # time
n = 10 # 반복 횟수

r = [r_0] # 3중 리스트: 전체 시계열 - object n - 요소
v = [v_0] # 3중 리스트: 전체 시계열 - object n - 요소
for i in range(n):
  r.append([])
  v.append([])

  result = RK4(t, r[-2], v[-2], h)
  r[-1] = result[0]
  v[-1] = result[1]